# Sistema de Detección de Quejas y Soporte Automático para E-commerce

## Importación de librerías

In [39]:
import pandas as pd
import gradio as gr

In [40]:
pd.set_option('display.max_colwidth', None)

In [41]:
from google.colab import userdata
import os

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [42]:
from difflib import get_close_matches

In [43]:
from google import genai
from google.genai import types

cliente = genai.Client(api_key=GOOGLE_API_KEY)

In [44]:
MODEL_ID = "gemini-2.0-flash-lite" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

## Dataset Simulado

In [45]:
# --- 1. Dataset simulado ---
tickets = [
    {"mensaje": "La cafetera llegó hecha pelota, una vergüenza. ¿Quién embala estas cosas?", "sentimiento": "Negativo", "categoria": "Producto defectuoso"},
    {"mensaje": "Todo perfecto, la entrega fue rapidísima. Gracias por la atención.", "sentimiento": "Positivo", "categoria": "Elogio"},
    {"mensaje": "¿Tienen en stock el aire acondicionado BGH de 3000 frigorías?", "sentimiento": "Neutro", "categoria": "Consulta de stock"},
    {"mensaje": "Che, ¿dónde está mi compra? Hace mil que la pagué y no aparece nada.", "sentimiento": "Negativo", "categoria": "Pedido no entregado"},
    {"mensaje": "Me cobraron dos veces el mismo pedido, arreglen eso urgente.", "sentimiento": "Negativo", "categoria": "Problema con pago"},
    {"mensaje": "El horno llegó justo a tiempo para el cumple, ¡genios!", "sentimiento": "Positivo", "categoria": "Elogio"},
    {"mensaje": "No puedo entrar a mi cuenta desde ayer, me tira error 503.", "sentimiento": "Negativo", "categoria": "Problema de acceso"},
    {"mensaje": "Necesito la factura del pedido #90876. ¿Dónde la encuentro?", "sentimiento": "Neutro", "categoria": "Consulta de facturación"},
    {"mensaje": "Pedí una tablet negra y me mandaron la blanca, ¿cómo lo soluciono?", "sentimiento": "Negativo", "categoria": "Error en el pedido"},
    {"mensaje": "No me contestan hace días, ¿hay alguien del otro lado?", "sentimiento": "Negativo", "categoria": "Falta de atención"},
    {"mensaje": "Gracias por el excelente servicio, volveré a comprar.", "sentimiento": "Positivo", "categoria": "Elogio"},
    {"mensaje": "¿Cuándo reponen el celular Motorola G200? Estoy esperando.", "sentimiento": "Neutro", "categoria": "Consulta de stock"},
    {"mensaje": "Ya pasaron 10 días y el paquete no llega, ¿qué onda?", "sentimiento": "Negativo", "categoria": "Retraso en envío"},
    {"mensaje": "Felicitaciones, el empaque estaba impecable y llegó antes de lo previsto.", "sentimiento": "Positivo", "categoria": "Elogio"},
    {"mensaje": "¿Cómo hago para descargar la factura de mi última compra?", "sentimiento": "Neutro", "categoria": "Consulta de facturación"}
]

In [46]:
df_tickets = pd.DataFrame(tickets)

## Clasificación Automática (Sentimiento + Categoría)

In [47]:
# --- 2. Etiquetas para clasificación de categoría ---
etiquetas = [
    "Producto defectuoso",
    "Pedido no entregado",
    "Retraso en envío",
    "Problema con pago",
    "Error en el pedido",
    "Problema de acceso",
    "Falta de atención",
    "Consulta de stock",
    "Elogio",
    "Consulta de facturación"
]

In [48]:
# --- 3. Función para normalizar la salida del modelo para sentimiento ---
def normalizar_sentimiento(valor):
    valor = valor.lower().strip()
    if "positiv" in valor:
        return "Positivo"
    elif "negativ" in valor:
        return "Negativo"
    else:
        return "Neutro"

In [49]:
# --- 4. Función para normalizar la salida del modelo para categoría usando fuzzy match ---
def normalizar_categoria(valor, etiquetas):
    match = get_close_matches(valor.strip(), etiquetas, n=1, cutoff=0.6)
    return match[0] if match else "Categoría desconocida"

In [50]:
# --- 5. Función de clasificación de sentimiento usando Gemini API (u otro LLM) ---
def clasificar_sentimiento(row):
    mensaje = row['mensaje']
    prompt = f"""
Clasificá este mensaje del cliente en una sola palabra: Positivo, Negativo o Neutro.
Devolvé solo la palabra, sin comillas ni explicaciones.

Mensaje: {mensaje}
"""
    respuesta = cliente.models.generate_content(
        model=MODEL_ID,
        contents=[prompt]
    )
    return normalizar_sentimiento(respuesta.text.strip())

In [51]:
# --- 6. Función de clasificación de categoría usando Zero-Shot ---
def clasificar_categoria(row):
    mensaje = row['mensaje']
    prompt = f"""
Clasificá el siguiente mensaje del cliente eligiendo solo una de estas categorías exactas, sin explicar:
{', '.join(etiquetas)}.

Devolvé únicamente una de las categorías anteriores, sin comillas ni texto adicional.

Mensaje: {mensaje}
"""
    respuesta = cliente.models.generate_content(
        model=MODEL_ID,
        contents=[prompt]
    )
    return normalizar_categoria(respuesta.text.strip(), etiquetas)

In [52]:
# --- 7. Aplicamos las funciones al DataFrame ---
df_tickets['sentimiento_pred'] = df_tickets.apply(clasificar_sentimiento, axis=1)
df_tickets['categoria_pred'] = df_tickets.apply(clasificar_categoria, axis=1)

In [ ]:
# --- 8. Evaluación: comparamos con etiquetas manuales ---
df_tickets['sentimiento_match'] = df_tickets['sentimiento'] == df_tickets['sentimiento_pred']
df_tickets['categoria_match'] = df_tickets['categoria'] == df_tickets['categoria_pred']

In [54]:
# --- 9. Métricas de precisión ---
print(" Accuracy sentimiento:", round(df_tickets['sentimiento_match'].mean()*100, 2), "%")
print(" Accuracy categoría:", round(df_tickets['categoria_match'].mean()*100, 2), "%")

 Accuracy sentimiento: 100.0 %
 Accuracy categoría: 93.33 %


In [55]:
# --- 10. Mostrar los errores para análisis ---
errores = df_tickets[(~df_tickets['sentimiento_match']) | (~df_tickets['categoria_match'])]
print("\n Casos con discrepancias:\n")
print(errores[['mensaje', 'sentimiento', 'sentimiento_pred', 'categoria', 'categoria_pred']])


 Casos con discrepancias:

                                                 mensaje sentimiento  \
12  Ya pasaron 10 días y el paquete no llega, ¿qué onda?    Negativo   

   sentimiento_pred         categoria       categoria_pred  
12         Negativo  Retraso en envío  Pedido no entregado  


Tras aplicar un modelo de lenguaje grande (LLM) para la clasificación automática de sentimiento y categoría sobre un dataset simulado de 15 tickets de soporte al cliente, se obtuvieron los siguientes resultados:

* Precisión en sentimiento: 100%

* Precisión en categorías: 93,33%

Estos resultados reflejan un desempeño muy positivo del modelo, especialmente en la detección del tono emocional de los mensajes, incluso en presencia de jerga coloquial argentina (ej. "hecha pelota", "qué onda", "che").

### Análisis de la discrepancia detectada

El único caso de discordancia fue el siguiente:

* Mensaje: “Ya pasaron 10 días y el paquete no llega, ¿qué onda?”
* Etiqueta manual: Retraso en envío
* Predicción del modelo: Pedido no entregado

Si bien ambas categorías están relacionadas, la elección del modelo sugiere una interpretación más binaria (“no entregado” vs. “retrasado”). Esta confusión es comprensible, ya que el límite entre ambas categorías puede ser ambiguo desde el lenguaje natural, especialmente sin contexto adicional (por ejemplo, un historial del pedido).

### Algunas conclusiones

El modelo mostró alta capacidad de comprensión del lenguaje informal y fue consistente con las etiquetas manuales en casi todos los casos.

Las discrepancias detectadas son semánticamente comprensibles y podrían reducirse aún más con un refinamiento en la definición de las categorías o ejemplos adicionales en los prompts.

Este tipo de sistema demuestra una buena viabilidad para automatizar la clasificación inicial de tickets, especialmente como herramienta de triage para priorizar y derivar consultas.

### Limitaciones y desafíos

La categoría “Retraso en envío” vs. “Pedido no entregado” ilustra que algunas etiquetas requieren contexto temporal o logístico que el modelo no tiene por defecto.

Podría evaluarse el uso de modelos fine-tuned si se busca precisión aún mayor, o reformular la taxonomía para agrupar categorías similares.

## Extracción de Información Clave (NER o QA)

In [56]:
import time

MAX_RETRIES = 5  # Máximo intentos para reintentar después de error 429
BASE_DELAY = 2   # Segundos base para el backoff exponencial

In [57]:
# --- 1. Función para el manejo del error 429 (RESOURCE_EXHAUSTED)---
def llamada_api_con_retry(prompt):
    for intento in range(1, MAX_RETRIES + 1):
        try:
            respuesta = cliente.models.generate_content(
                model=MODEL_ID,
                contents=[prompt]
            )
            time.sleep(1)  # Pausa para evitar saturar API
            return respuesta.text.strip()
        except Exception as e:
            # Detectamos error de cuota agotada (429)
            if 'RESOURCE_EXHAUSTED' in str(e) or '429' in str(e):
                delay = BASE_DELAY * (2 ** (intento - 1))  # Backoff exponencial
                print(f"Error 429 detectado. Reintentando en {delay} segundos... (Intento {intento}/{MAX_RETRIES})")
                time.sleep(delay)
                continue
            else:
                # Otro error, lo propagamos o retornamos mensaje
                print(f"Error inesperado: {e}")
                return f"Error al procesar el texto: {e}"
    return "Error: Se agotaron los intentos por error 429."


In [58]:
# --- 2. Función para extraer entidades nombradas (NER simulado con prompt) ---
def analizar_entidades(row):
    mensaje = row['mensaje']
    prompt = f"""
Extraé todas las entidades nombradas en el siguiente mensaje y clasificálas:

CATEGORÍAS:
- PERSONA: Nombres de personas
- LUGAR: Ciudades, países, barrios, direcciones, lugares específicos
- ORGANIZACIÓN: Empresas, universidades, instituciones
- MISCELÁNEO: Otros nombres propios (productos, eventos, marcas)

FORMATO DE RESPUESTA:
[ENTIDAD] → [CATEGORÍA] → [BREVE EXPLICACIÓN]

Texto:
{mensaje}
"""
    return llamada_api_con_retry(prompt)

In [59]:
# --- 3. Función para responder preguntas específicas (QA) ---
def responder_preguntas(row):
    mensaje = row['mensaje']
    prompt = f"""Basado en el siguiente texto, respondé las siguientes preguntas en formato de lista:
Texto: {mensaje}

Preguntas:
- ¿Qué producto menciona el cliente?
- ¿Cuál es el problema principal del cliente?
- ¿Dónde ocurrió el problema?
- ¿Se menciona algún número de pedido?

Formato de respuesta:
[Producto: <respuesta>, Problema: <respuesta>, Ubicación: <respuesta>, Pedido: <respuesta>]
Si la información no está disponible, respondé "No especificado".
"""
    return llamada_api_con_retry(prompt)

## Generación de Respuesta Automática

In [60]:
# --- 1. Función para generar respuesta de atención al cliente ---
def respuesta_per(row):
    mensaje = row['mensaje']
    prompt = f"""Basado en el siguiente mensaje, redactá una respuesta del servicio al cliente. La respuesta tiene que ser
empática, debe reconocer el problema y sugerir el siguiente paso. No debe tener más de 4 líneas.

Texto: {mensaje}
"""
    return llamada_api_con_retry(prompt)

In [62]:
# --- 2. Aplicamos las funciones al DataFrame ---
df_tickets['entidades'] = df_tickets.apply(analizar_entidades, axis=1)
df_tickets['respuestas_qa'] = df_tickets.apply(responder_preguntas, axis=1)
df_tickets['respuesta_llm'] = df_tickets.apply(respuesta_per, axis=1)

In [63]:
# --- 6. Visualizamos resultados para verificar ---
for i, fila in df_tickets.iterrows():
    print(f"Ticket #{i+1}:")
    print(f"Mensaje: {fila['mensaje']}")
    print(f"Entidades extraídas:\n{fila['entidades']}")
    print(f"Respuestas a preguntas:\n{fila['respuestas_qa']}")
    print(f"Respuesta generada por LLM:\n{fila['respuesta_llm']}")
    print("-" * 50)

Ticket #1:
Mensaje: La cafetera llegó hecha pelota, una vergüenza. ¿Quién embala estas cosas?
Entidades extraídas:
Aquí están las entidades nombradas y sus categorías:

*   **Ninguna entidad nombrada identificada** → **Ninguna categoría** → El texto no contiene nombres propios de personas, lugares, organizaciones o misceláneos.
Respuestas a preguntas:
[Producto: Cafetera, Problema: La cafetera llegó dañada, Ubicación: No especificado, Pedido: No especificado]
Respuesta generada por LLM:
Lamentamos mucho que tu cafetera haya llegado dañada. Es una clara falta de cuidado.

Por favor, contacta a nuestro equipo de soporte técnico con fotos del daño y el número de pedido. 
Te asistiremos con la resolución a la brevedad.
--------------------------------------------------
Ticket #2:
Mensaje: Todo perfecto, la entrega fue rapidísima. Gracias por la atención.
Entidades extraídas:
El mensaje no contiene ninguna entidad nombrada.
Respuestas a preguntas:
[Producto: No especificado, Problema: No es

### Conclusiones:

* Las entidades nombradas se extraen correctamente en casos donde hay marcas, códigos o números de pedido.
Por ejemplo: “BGH” como organización, “#90876” como número de pedido, “Motorola G200” como misc.

* Para textos más informales o con lenguaje común (sin nombres propios), el modelo indica correctamente que no hay entidades nombradas.
Eso evita ruido en el análisis.

* Las respuestas del LLM se ven empáticas y contextuales, con reconocimiento del problema y sugerencias claras para el cliente.
El tono es adecuado y respetuoso, lo que mejora la experiencia.

* La extracción de info clave (producto, problema, ubicación, pedido) es precisa y útil para automatizar tareas de clasificación o asignación.

* Se identifican bien casos sin datos específicos, lo que permite pedir más información cuando hace falta.

## Interfaz Interactiva con Gradio

In [64]:
# --- 1. Función principal para interfaz ---

def procesar_ticket(mensaje):
    row = {"mensaje": mensaje}
    try:
        sentimiento = clasificar_sentimiento(row)
        categoria = clasificar_categoria(row)
        ner = analizar_entidades(row)
        qa = responder_preguntas(row)
        respuesta = respuesta_per(row)
    except Exception as e:
        error_msg = f"Error al procesar el mensaje: {str(e)}"
        return error_msg, error_msg, error_msg, error_msg, error_msg

    return sentimiento, categoria, ner, qa, respuesta

In [65]:
# --- 2. Ejemplos ---

ejemplos = [
    ["La cafetera llegó hecha pelota, una vergüenza. ¿Quién embala estas cosas?"],
    ["¿Cuándo reponen el celular Motorola G200? Estoy esperando."],
    ["Gracias, el paquete llegó a tiempo y perfecto."],
    ["No puedo entrar a mi cuenta desde ayer, me tira error 503."],
    ["Pedí una tablet negra y me mandaron la blanca, ¿cómo lo soluciono?"],
]

In [66]:
# --- 3. Interfaz Gradio ---

iface = gr.Interface(
    fn=procesar_ticket,
    inputs=gr.Textbox(lines=4, label="📨 Mensaje del cliente"),
    outputs=[
        gr.Textbox(label="🎭 Sentimiento"),
        gr.Textbox(label="📂 Categoría"),
        gr.Textbox(label="🧠 Entidades extraídas"),
        gr.Textbox(label="❓ Preguntas clave (QA)"),
        gr.Textbox(lines=4, label="🤖 Respuesta generada"),
    ],
    examples=ejemplos,
    title="🤝 Asistente de Atención al Cliente",
    description="Ingresá un mensaje de cliente y el sistema analizará sentimiento, categoría, entidades, preguntas clave y generará una respuesta empática.",
    theme="soft"
)

In [ ]:
iface.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://87c031f05b5cb99b85.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Conclusiones

La implementación del Asistente de Atención al Cliente Inteligente ha demostrado ser exitosa y funcional, integrando análisis automatizado de lenguaje natural con una interfaz intuitiva. El sistema es capaz de:

* Comprender e interpretar mensajes de clientes:

    - Detecta el sentimiento emocional del mensaje (positivo, negativo, neutro).

    - Clasifica automáticamente el mensaje dentro de categorías de atención (reclamos, consultas, agradecimientos, etc.).

    - Extrae entidades clave como productos, marcas, números de pedido o errores técnicos.

    - Responde preguntas fundamentales con base en el texto.

    - Genera una respuesta empática, concisa y orientada a la acción, lista para ser enviada.

* Beneficios concretos:

    - Acelera el proceso de triage de tickets.

    - Mejora la consistencia en las respuestas al cliente.

    - Permite detectar automáticamente casos urgentes o repetitivos.

    - Aporta estructura y comprensión semántica a mensajes no estructurados.

* Interfaz interactiva:

    - Fácil de usar para operadores de atención al cliente o supervisores.

    - Incluye ejemplos predefinidos y puede ser extendida con nuevos casos.

    - Corre localmente o puede ser desplegada en plataformas web.

Si bien este tipo de soluciones puede mejorar significativamente la eficiencia en la gestión de tickets, consideramos que es importante destacar que:

* No reemplaza completamente la intervención humana.

* Los modelos pueden cometer errores de interpretación, especialmente en textos ambiguos, irónicos o culturalmente específicos.

* Los casos delicados o emocionales requieren empatía real y toma de decisiones humanas.

* El sistema debe ser utilizado como una herramienta de apoyo, que agiliza el trabajo, pero no elimina la necesidad de criterio profesional.